In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
import nibabel as nib
from tqdm import tqdm
import cv2 as cv
from scipy import ndimage
import time
import pandas as pd
import json
from utils import *
from Register_utils import *
from scipy.ndimage import binary_fill_holes
from nilearn import image
from scipy.ndimage import binary_erosion, generate_binary_structure
import seaborn as sns

In [2]:
def Smooth4GM(gmpath,fwhm,threshold):
    # 读取NIfTI文件
    image = sitk.ReadImage(gmpath)

    # 高斯平滑
    smoothed_image = sitk.SmoothingRecursiveGaussian(image, fwhm)

    # 二值化
    binary_image = sitk.BinaryThreshold(smoothed_image, lowerThreshold=threshold, upperThreshold=float('inf'))

    # 将SimpleITK图像转换为NumPy数组
    binary_array = sitk.GetArrayFromImage(binary_image)

    return binary_array

def Erosion4WM(wmpath,radius,threshold):
    # 读取NIfTI文件
    wm_arr = sitk.GetArrayFromImage(sitk.ReadImage(wmpath))
    wm_mask = wm_arr > threshold
    structure = generate_binary_structure(3, 1)  # 3D structure with connectivity 2
    binary_array = binary_erosion(wm_mask, structure, iterations=radius)

    return binary_array

def CalMeanStdofCBF(cbf_arr,mask_arr):
    intersection_values = cbf_arr[(cbf_arr > 1) & (mask_arr > 0)]
    mean = np.mean(intersection_values)
    std = np.std(intersection_values)
    return mean,std

def CalContrastofGMWM(meanofgm,meanofwm):
    contrast = meanofgm/meanofwm
    return contrast

def CalCOV(cbf_arr):
    brain = cbf_arr[cbf_arr > 0]
    mean = np.mean(brain)
    std = np.std(brain)
    cov = std/mean
    return cov

def CalSNR(mean,std):
    snr = mean / std
    return snr

def CalpSNR(cbf_arr,mask_arr):
    roi = cbf_arr[(mask_arr > 0) & (cbf_arr > 1)]
    psnr = 10 * np.log10(np.max(roi)**2 / np.mean((roi - np.mean(roi))**2))
    return psnr

def CalAsymmetry(cbf_arr):
    asymmetry = np.mean(np.abs(np.flip(cbf_arr, axis=2) - cbf_arr))/np.mean((0.5*(np.flip(cbf_arr, axis=2) + cbf_arr)))
    return asymmetry

def overlap_Histogram(img_list,color_list,vmin= 0, vmax=120):
    for i, (img, color) in enumerate(zip(img_list,color_list)):
        nums = img[img>1][:]
        df = pd.DataFrame(nums, columns=['intensity'])
        plt.xlim(vmin,vmax)
        plt.ylim(0,0.08)
        sns.kdeplot(data=df, x='intensity', alpha=0.3, color=color)

In [ ]:
# distribution before qc
df = pd.read_csv('xxxx.csv')
plt.subplots(nrows=1, ncols=2, figsize=(15,5))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
for index, row in tqdm(df.iterrows(), total=len(df)):
    T1path = row['T1wpath']
    CBFpath = row['CBFpath']
    c1path = T1path.replace('masked_T1w_toMNI','masked_c1_toMNI')
    c2path = T1path.replace('masked_T1w_toMNI','masked_c2_toMNI')
    if os.path.exists(CBFpath) and os.path.exists(c1path):
        gm_arr = Smooth4GM(c1path,3,0.6)
        # Plot3D(gm_arr,vmin=0,vmax=1,layout=0)
        wm_arr = Erosion4WM(c2path,3,0.5)
        # Plot3D(wm_arr,vmin=0,vmax=1,layout=0)
        cbf_arr = sitk.GetArrayFromImage(sitk.ReadImage(CBFpath))
        roi_gm = cbf_arr[(gm_arr > 0) & (cbf_arr > 1)]
        roi_wm = cbf_arr[(wm_arr > 0) & (cbf_arr > 1)]
        plt.subplot(1,2,1)
        plt.title('GM CBF Distribution', fontsize=16)
        overlap_Histogram([roi_gm], 'r',vmin=0, vmax=180)
        plt.subplot(1,2,2)
        plt.title('WM CBF Distribution', fontsize=16)
        overlap_Histogram([roi_wm], 'b',vmin=0, vmax=100)
plt.savefig('gmwm_distribution_before_QC.png')

In [ ]:
# fill in the qc csv
df = pd.read_csv('/home/data/ext/Resources/PTBP/PTBP_processed240110.csv')
df['asy'] = np.nan
df['Mean_of_GM'] = np.nan
df['Mean_of_WM'] = np.nan
df['Std_of_GM'] = np.nan
df['Std_of_WM'] = np.nan
df['Contrast'] = np.nan
df['COV'] = np.nan
df['SNR_of_GM'] = np.nan
df['SNR_of_WM'] = np.nan
df['pSNR_of_GM'] = np.nan
df['pSNR_of_WM'] = np.nan
df['Mean_of_GM_anat'] = np.nan
df['Mean_of_WM_anat'] = np.nan
# rootpath = '/home/data/ext/Resources/QTAB/All/'
for index, row in tqdm(df[0:].iterrows(), total=len(df[0:])):
    # mni
    T1path = row['T1wpath']
    CBFpath = row['CBFpath']
    c1path = T1path.replace('T1w_toMNI','masked_c1_toMNI')
    c2path = T1path.replace('T1w_toMNI','masked_c2_toMNI')
    # native
    CBFpath_native = T1path.replace('MNI/T1w_toMNI.nii','CBF/CBF_toT1w.nii')
    if not os.path.exists(CBFpath_native):
        CBFpath_native = T1path.replace('MNI/T1w_toMNI.nii','PCASL/CBF_toT1w.nii')
    c1path_native = T1path.replace('MNI/T1w_toMNI.nii','Anatomy/c1T1w_N4.nii')
    c2path_native = T1path.replace('MNI/T1w_toMNI.nii','Anatomy/c2T1w_N4.nii')
    if os.path.exists(CBFpath) and os.path.exists(c1path) and os.path.exists(CBFpath_native) and os.path.exists(c2path_native):
        try:
            gm_arr = Erosion4WM(c1path,1,0.5)
            wm_arr = Erosion4WM(c2path,3,0.5)
            cbf_arr = sitk.GetArrayFromImage(sitk.ReadImage(CBFpath))
            
            gm_arr_native = Erosion4WM(c1path_native,1,0.5)
            wm_arr_native = Erosion4WM(c2path_native,3,0.5)
            cbf_arr_native = sitk.GetArrayFromImage(sitk.ReadImage(CBFpath_native))
            
            # Plot3D(gm_arr,vmin=0,vmax=1,layout=0)
            # Plot3D(wm_arr,vmin=0,vmax=1,layout=0)
            # Plot3D(gm_arr_native,vmin=0,vmax=1,layout=1)
            # Plot3D(wm_arr_native,vmin=0,vmax=1,layout=1)
            
            asy = CalAsymmetry(cbf_arr)
            meanofgm,stdofgm = CalMeanStdofCBF(cbf_arr,gm_arr)
            meanofwm,stdofwm = CalMeanStdofCBF(cbf_arr,wm_arr)
            contrast = CalContrastofGMWM(meanofgm,meanofwm)
            cov = CalCOV(cbf_arr)
            snrofGM = CalSNR(meanofgm,stdofgm)
            snrofWM = CalSNR(meanofwm,stdofwm)
            psnrofGM = CalpSNR(cbf_arr,gm_arr)
            psnrofWM = CalpSNR(cbf_arr,wm_arr)
            meanofgm_native, _ = CalMeanStdofCBF(cbf_arr_native,gm_arr_native)
            meanofwm_native, _ = CalMeanStdofCBF(cbf_arr_native,wm_arr_native)
            # print(meanofwm_native)

            df.at[index, 'asy'] = asy
            df.at[index, 'Mean_of_GM'] = meanofgm
            df.at[index, 'Mean_of_WM'] = meanofwm
            df.at[index, 'Std_of_GM'] = stdofgm
            df.at[index, 'Std_of_WM'] = stdofwm
            df.at[index, 'Contrast'] = contrast
            df.at[index, 'COV'] = cov
            df.at[index, 'SNR_of_GM'] = snrofGM
            df.at[index, 'SNR_of_WM'] = snrofWM
            df.at[index, 'pSNR_of_GM'] = psnrofGM
            df.at[index, 'pSNR_of_WM'] = psnrofWM
            # df.at[index, 'Mean_of_GM_native'] = meanofgm_native
            # df.at[index, 'Mean_of_WM_native'] = meanofwm_native
            df.at[index, 'Mean_of_GM_anat'] = meanofgm_native
            df.at[index, 'Mean_of_WM_anat'] = meanofwm_native
        except:
            print('Wrong with',CBFpath)
df.to_csv('qc.csv', index=False)

In [ ]:
# distribution after qc
df = pd.read_csv('xxxx.csv')
df = df[df['QC'] == 1]
plt.subplots(nrows=1, ncols=2, figsize=(15,5))
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
for index, row in tqdm(df.iterrows(), total=len(df)):
    T1path = row['T1wpath']
    CBFpath = row['CBFpath']
    c1path = T1path.replace('masked_T1w_toMNI','masked_c1_toMNI')
    c2path = T1path.replace('masked_T1w_toMNI','masked_c2_toMNI')
    if os.path.exists(CBFpath) and os.path.exists(c1path):
        gm_arr = Smooth4GM(c1path,3,0.6)
        # Plot3D(gm_arr,vmin=0,vmax=1,layout=0)
        wm_arr = Erosion4WM(c2path,3,0.5)
        # Plot3D(wm_arr,vmin=0,vmax=1,layout=0)
        cbf_arr = sitk.GetArrayFromImage(sitk.ReadImage(CBFpath))
        roi_gm = cbf_arr[(gm_arr > 0) & (cbf_arr > 1)]
        roi_wm = cbf_arr[(wm_arr > 0) & (cbf_arr > 1)]
        plt.subplot(1,2,1)
        plt.title('GM CBF Distribution', fontsize=16)
        overlap_Histogram([roi_gm], 'r',vmin=0, vmax=180)
        plt.subplot(1,2,2)
        plt.title('WM CBF Distribution', fontsize=16)
        overlap_Histogram([roi_wm], 'b',vmin=0, vmax=100)
plt.savefig('gmwm_distribution_after_QC.png')